# Vectorstores and Embeddings

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("docs/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("docs/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("docs/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_spilliter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

In [4]:
splits = text_spilliter.split_documents(docs)

In [5]:
len(splits)

208

## Embeddings

In [6]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
sentence1 = "i like |dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [8]:
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)

In [9]:
import numpy as np

In [10]:
np.dot(embedding1, embedding2)

np.float64(0.9476759949524862)

In [11]:
np.dot(embedding1, embedding3)

np.float64(0.7691756383788506)

In [12]:
np.dot(embedding2, embedding3)

np.float64(0.759054062979165)

## Vectorstores

In [13]:
from langchain.vectorstores import Chroma

In [14]:
persist_directory = 'docs/chroma/'

In [15]:
vectordb = Chroma.from_documents(documents=splits,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [16]:
print(vectordb._collection.count())

208


## Similarity search

In [17]:
question = "is there an email i can ask for help"

In [18]:
docs = vectordb.similarity_search(question, k=3)

In [19]:
len(docs)

3

In [20]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

## Failure modes

In [21]:
question = "what did they say about matlab?"

In [22]:
docs = vectordb.similarity_search(question, k=5)

In [23]:
docs[0]

Document(metadata={'total_pages': 22, 'page': 8, 'title': '', 'page_label': '9', 'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:23-07:00', 'creationdate': '2008-07-11T11:25:23-07:00', 'source': 'docs/MachineLearning-Lecture01.pdf', 'producer': 'Acrobat Distiller 8.1.0 (Windows)'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB li

In [24]:
docs[1]

Document(metadata={'author': '', 'page_label': '9', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 22, 'source': 'docs/MachineLearning-Lecture01.pdf', 'title': '', 'creationdate': '2008-07-11T11:25:23-07:00', 'page': 8, 'moddate': '2008-07-11T11:25:23-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB li

In [25]:
question = "what did they say about regression in the third lecture?"

In [26]:
docs = vectordb.similarity_search(question, k=5)

In [27]:
for doc in docs:
    print(doc.metadata)

{'author': '', 'source': 'docs/MachineLearning-Lecture03.pdf', 'title': '', 'total_pages': 16, 'creationdate': '2008-07-11T11:25:03-07:00', 'moddate': '2008-07-11T11:25:03-07:00', 'page_label': '1', 'page': 0, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2'}
{'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationdate': '2008-07-11T11:25:05-07:00', 'page': 2, 'title': '', 'moddate': '2008-07-11T11:25:05-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'source': 'docs/MachineLearning-Lecture02.pdf', 'total_pages': 18, 'author': '', 'page_label': '3'}
{'source': 'docs/MachineLearning-Lecture02.pdf', 'moddate': '2008-07-11T11:25:05-07:00', 'page_label': '18', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 18, 'page': 17, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'title': '', 'author': '', 'creationdate': '2008-07-11T11:25:05-07:00'}
{'moddate': '2008-07-11T11:25:23-07:00', 'total_pages': 22, 'source': 'docs/MachineLearning-Lectur

In [28]:
print(docs[4].page_content)

into his office and he said, "Oh, professor, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's helped me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "Wow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data networks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutorial in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical thing is the discussion sections. So discussion 
sections will be taught by the TAs, and attendance at discussion sections is o